In [ ]:
using Revise
using AllenCahn
using Plots; gr()
using LinearAlgebra
using LaTeXStrings
using Dierckx

# Testing the Package

We start by running some basic tests of the package (checking to make sure constructors and functions run as designed, and checking a test problem). All of these tests indicate that the package was coded properly, as the test unit passes all instances of the problem (with different timestepping and initial condition combinations).

In [ ]:
# Construct the problem with Neumann BCs
a = 0; b = 1
ϵ = 0.1
nₓ = 9
nₜ = 100; t_max = 1
left_bc = NeumannBC(0); right_bc = NeumannBC(0)
u₀(x) = 1.
problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀);

In [ ]:
# Check that the problem constructor works
println("a: $(problem.a), b: $(problem.b), ϵ: $(problem.ϵ), Δx: $(problem.Δx) Δt: $(problem.Δt)")
println("mesh: $(problem.x)")

In [ ]:
# Assembly and check
assemble_system!(problem);
println("A: $(problem.A)")
println("RHS: $(problem.rhs)")

In [ ]:
# Solve problem and plot solution vs. expected result (solve with Backward Euler)
t, u = solve(problem, BackwardEulerMethod())
u_true(x) = 1.
plot(problem.x, u_true.(problem.x), label = "True Solution", title = "Solution with Neumann BCs + BE", ylims = (0, 2))
scatter!(problem.x, u[end], label = "Numerical Solution")
xlabel!("x"); ylabel!("u(x, 1)")

In [ ]:
# Construct and solve problem with Periodic BCs
a = 0; b = 1
ϵ = 0.1
nₓ = 9
nₜ = 100; t_max = 1
left_bc = PeriodicBC(); right_bc = PeriodicBC()
u₀(x) = 1.
problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀);

assemble_system!(problem);
t, u = solve(problem, BackwardEulerMethod())
plot(problem.x, u_true.(problem.x), label = "True Solution", title = "Solution with Periodic BCs + BE", ylims = (0, 2))
scatter!(problem.x, u[end], label = "Numerical Solution")
xlabel!("x"); ylabel!("u(x, 1)")

In [ ]:
# Construct and solve problem with Neumann BCs and Crank-Nicolson
a = 0; b = 1
ϵ = 0.1
nₓ = 9
nₜ = 100; t_max = 1
left_bc = NeumannBC(0); right_bc = NeumannBC(0)
u₀(x) = 1.
problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀);

assemble_system!(problem);
t, u = solve(problem, CrankNicolsonMethod())
plot(problem.x, u_true.(problem.x), label = "True Solution", title = "Solution with Neumann BCs + CN", ylims = (0, 2))
scatter!(problem.x, u[end], label = "Numerical Solution")
xlabel!("x"); ylabel!("u(x, 1)")

In [ ]:
# Construct and solve problem with Periodic BCs and Crank-Nicolson
a = 0; b = 1
ϵ = 0.1
nₓ = 9
nₜ = 100; t_max = 1
left_bc = PeriodicBC(); right_bc = PeriodicBC()
u₀(x) = 1.
problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀);

assemble_system!(problem);
t, u = solve(problem, CrankNicolsonMethod())
plot(problem.x, u_true.(problem.x), label = "True Solution", title = "Solution with Periodic BCs + CN", ylims = (0, 2))
scatter!(problem.x, u[end], label = "Numerical Solution")
xlabel!("x"); ylabel!("u(x, 1)")

In [ ]:
# Use a test to check that the package passes
using Pkg
Pkg.test("AllenCahn")

# Computing Solutions and Assessing Convergence

For a problem that we do not know the exact solution for (i.e., with $u_{0} = \sin(2\pi x)$ and periodic boundary conditions), we want to test the convergence in space and time. We start by computing a high resolution solution in space and time and comparing it against a solution with a more reasonable resolution:

In [ ]:
# Compute a high-reslution solution for u₀ = sin(2πx)
a = 0; b = 1
ϵ = 0.1
nₓ_hi = Int(1e2 - 1)
nₜ_hi = Int(1e4); t_max = 1
left_bc = PeriodicBC(); right_bc = PeriodicBC()
u₀(x) = sin(2*π*x)

problem_hi = AllenCahnProblem1D(a, b, ϵ, nₓ_hi, nₜ_hi, t_max, left_bc, right_bc, u₀)
assemble_system!(problem_hi)
tsm = BackwardEulerMethod()
t_hi, u_hi = solve(problem_hi, tsm)
plot(problem_hi.x, u_hi[end], label = "uₕ", title = "High Resolution Solution vs. Low Resolution Solution")

# Compute a regular-resolution solution
nₓ = Int(1e1 - 1)
nₜ = Int(1e2); t_max = 1

problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀)
assemble_system!(problem)
t, u = solve(problem, tsm)
scatter!(problem.x, u[end], label = "uₗ")
xlabel!("x"); ylabel!("u(x, 1)")

Now we check the convergence in time for each of the timestepping methods (backwards Euler and Crank-Nicolson):

In [ ]:
# Assessment of convergence rate in time with Backward Euler
a = 0; b = 1
ϵ = 0.1
nₓ = 99; nₜ_hi = 100_000; t_max = 1.
left_bc = PeriodicBC(); right_bc = PeriodicBC()
u₀(x) = sin(2*π*x)

problem_hi = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ_hi, t_max, left_bc, right_bc, u₀)
assemble_system!(problem_hi)
t_hi, u_hi = solve(problem_hi, BackwardEulerMethod())

nₜ_values = [10^n for n ∈ 1:4]
Δt_values = [1/nₜ for nₜ in nₜ_values]
error_values = []
for nₜ ∈ nₜ_values
    problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀)
    assemble_system!(problem)
    t, u = solve(problem, BackwardEulerMethod())
    error = norm(u[end] .- u_hi[end], Inf)
    push!(error_values, error)
end

scatter(Δt_values, error_values, label = "Infinity norm error",
        title = "Rate of Convergence in Time with BE", xscale = :log10, yscale = :log10, legend = :bottomright)
plot!(Δt_values, Δt_values, label = "Linear in Δt")
xlabel!("Δt"); ylabel!("Error")

In [ ]:
# Assessment of convergence rate in time with Crank-Nicolson
a = 0; b = 1
ϵ = 0.1
nₓ = 99; nₜ_hi = 200_000; t_max = 1.
left_bc = PeriodicBC(); right_bc = PeriodicBC()
u₀(x) = sin(2*π*x)

problem_hi = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ_hi, t_max, left_bc, right_bc, u₀)
assemble_system!(problem_hi)
t_hi, u_hi = solve(problem_hi, CrankNicolsonMethod())

nₜ_values = [10^n for n ∈ 1:4]
Δt_values = [1/nₜ for nₜ in nₜ_values]
error_values = []
for nₜ ∈ nₜ_values
    problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀)
    assemble_system!(problem)
    t, u = solve(problem, CrankNicolsonMethod())
    error = norm(u[end] .- u_hi[end], Inf)
    push!(error_values, error)
end

scatter(Δt_values, error_values, label = "Infinity norm error",
        title = "Rate of Convergence in Time with CN", xscale = :log10, yscale = :log10, legend = :bottomright)
plot!(Δt_values, Δt_values, label = "Linear in Δt")
xlabel!("Δt"); ylabel!("Error")

We find that Backwards Euler provides linear-in-time convergence, and Crank-Nicolson provides linear-in-time convergence. Backwards Euler's theoretical prediction is linear in time, so this is expected behavior, but the Crank-Nicolson algorithm is supposed to be quadratic in time, so the performance here is worse than expected. Next, we assess the convergence in space:

In [ ]:
# Assessment of convergence rate in space with Finite Differences
a = 0; b = 1
ϵ = 0.1
nₓ_hi = 1_000; nₜ = 10_000; t_max = 1.
left_bc = PeriodicBC(); right_bc = PeriodicBC()
u₀(x) = sin(2*π*x)

problem_hi = AllenCahnProblem1D(a, b, ϵ, nₓ_hi, nₜ, t_max, left_bc, right_bc, u₀)
assemble_system!(problem_hi)
t_hi, u_hi = solve(problem_hi, CrankNicolsonMethod())
u_hi_spl = Spline1D(problem_hi.x, u_hi[end], k=1);

nₓ_values = 50:200:850
Δx_values = [1/nₓ for nₓ in nₓ_values]
error_values = []
for nₓ ∈ nₓ_values
    problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀)
    assemble_system!(problem)
    t, u = solve(problem, CrankNicolsonMethod())
    error = norm(u[end] .- u_hi_spl.(problem.x), Inf)
    push!(error_values, error)
end

scatter(Δx_values, error_values, label = "Infinity norm error",
        title = "Rate of Convergence in Space with FD", xscale = :log10, yscale = :log10, legend = :bottomright)
xticks!([2e-3, 4e-3, 1e-2])
xlabel!("Δx"); ylabel!("Error")
plot!(Δx_values, Δx_values.^2, label = "Quadratic in Δx")

We can see that the convergence in space is quadratic, which we expect from using finite differences for the second derivative and having periodic boundary conditions. Finally, we integrate $u_{0} = \sin(4\pi x)$ to $t = 10$:

In [ ]:
a = 0; b = 1
ϵ = 0.1
nₓ = 100; nₜ = 10_000; t_max = 10.
left_bc = PeriodicBC(); right_bc = PeriodicBC()
u₀(x) = sin(4*π*x)

problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀)
assemble_system!(problem)
t, u = solve(problem, CrankNicolsonMethod())

u = permutedims(reshape(hcat(u...), (length(u[1]), length(u))))
contourf(problem.x, t, u, xlabel = "x", ylabel = "t", title = "Evolution of Solution")